In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select , WebDriverWait
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
# Initialize the Edge WebDriver
driver = webdriver.Edge()

# Open the target page
driver.get('https://sjp.moj.gov.sa/Filter?isFilterButtonClicked=True')
driver.maximize_window()
time.sleep(1)

# Select value '483' from the first dropdown
dropdown = driver.find_element(By.ID, 'darga1')
radius = Select(dropdown)
radius.select_by_value('1')
time.sleep(2)

# Select value '529' from the second dropdown
dropdown2 = driver.find_element(By.ID, 'darga2')
radius2 = Select(dropdown2)
radius2.select_by_value('444')
time.sleep(2)

page_size_dropdown = driver.find_element(By.ID, 'PageSize')
page_size_select = Select(page_size_dropdown)
page_size_select.select_by_value('100')
time.sleep(2)

In [ ]:
for k in range(206):
    links = driver.find_elements(By.XPATH, '//a[@class="main2-color"]')

    with open('Cases__links.csv', 'a', encoding='utf-8') as file:
        for link in links:
            try:
                href_value = link.get_attribute("href")
                if href_value is not None:
                    file.write(href_value + '\n')
            except StaleElementReferenceException:
                print("StaleElementReferenceException encountered. Refetching the element.")
                links = driver.find_elements(By.XPATH, '//a[@class="main2-color"]')
                href_value = link.get_attribute("href")
                if href_value is not None:
                    file.write(href_value + '\n')

    try:
        next_button = WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@rel="next"]'))
        )
        next_button.click()
        time.sleep(4)

    except NoSuchElementException:
        print("No next button found or reached the end.")
        break
    except ElementNotInteractableException:
        print("Next button not interactable.")
        break

## Links Cleaning

In [ ]:
df = pd.read_csv('Cases_links.csv')
df

,https://sjp.moj.gov.sa/
0,https://sjp.moj.gov.sa/Filter/AhkamDetails/13340
1,https://sjp.moj.gov.sa/Filter/AhkamDetails/13341
2,https://sjp.moj.gov.sa/Filter/AhkamDetails/13343
3,https://sjp.moj.gov.sa/Filter/AhkamDetails/13424
4,https://sjp.moj.gov.sa/Filter/AhkamDetails/13518
...,...
116470,https://sjp.moj.gov.sa/
116471,https://sjp.moj.gov.sa/AboutSite
116472,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...
116473,https://sjp.moj.gov.sa/ContactUs


In [16]:
unique_df = df.drop_duplicates()
print(unique_df)

                                 https://sjp.moj.gov.sa/
0       https://sjp.moj.gov.sa/Filter/AhkamDetails/13340
1       https://sjp.moj.gov.sa/Filter/AhkamDetails/13341
2       https://sjp.moj.gov.sa/Filter/AhkamDetails/13343
3       https://sjp.moj.gov.sa/Filter/AhkamDetails/13424
4       https://sjp.moj.gov.sa/Filter/AhkamDetails/13518
...                                                  ...
115743  https://sjp.moj.gov.sa/Filter/AhkamDetails/13601
115744  https://sjp.moj.gov.sa/Filter/AhkamDetails/13142
115745  https://sjp.moj.gov.sa/Filter/AhkamDetails/13144
115746  https://sjp.moj.gov.sa/Filter/AhkamDetails/13162
115747  https://sjp.moj.gov.sa/Filter/AhkamDetails/13163

[20193 rows x 1 columns]


In [17]:
unique_df.isnull().sum()

https://sjp.moj.gov.sa/    0
dtype: int64

In [18]:
filtered_df = unique_df[unique_df['https://sjp.moj.gov.sa/'].str.startswith('https://sjp.moj.gov.sa/Filter/AhkamDetails/')]
print(filtered_df)

                                 https://sjp.moj.gov.sa/
0       https://sjp.moj.gov.sa/Filter/AhkamDetails/13340
1       https://sjp.moj.gov.sa/Filter/AhkamDetails/13341
2       https://sjp.moj.gov.sa/Filter/AhkamDetails/13343
3       https://sjp.moj.gov.sa/Filter/AhkamDetails/13424
4       https://sjp.moj.gov.sa/Filter/AhkamDetails/13518
...                                                  ...
115743  https://sjp.moj.gov.sa/Filter/AhkamDetails/13601
115744  https://sjp.moj.gov.sa/Filter/AhkamDetails/13142
115745  https://sjp.moj.gov.sa/Filter/AhkamDetails/13144
115746  https://sjp.moj.gov.sa/Filter/AhkamDetails/13162
115747  https://sjp.moj.gov.sa/Filter/AhkamDetails/13163

[20188 rows x 1 columns]


In [19]:
filtered_df = filtered_df.rename(columns={'https://sjp.moj.gov.sa/': 'Links'}) 

In [ ]:
filtered_df.to_csv('cleaned_Cases_links.csv', index=False)